In [ ]:
!pip install torch

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

In [ ]:
!pip install bertopic[all] sentence-transformers umap-learn hdbscan

In [2]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import json

In [3]:
with open("/home/shashank/Code/MiniProject/Query4Code.jsonl", "r") as f:
  for line in f:
    data = [json.loads(line) for line in f]
print(len(data))
# not sure what to do here
# should we use both code and query or only code
docs = [f"code:{item['code']} query:{item['query']}"for item in data]

237199


In [ ]:
model = BERTopic(    
    embedding_model="all-MiniLM-L6-v2",   # Efficient embedding model for quick computation
    umap_model=UMAP(
        n_neighbors=10,                  # Lower neighbors for tighter clusters
        n_components=5,                  # Dimensionality reduction
        min_dist=0.1,                    # Slight separation between clusters
        metric='cosine'                  # Cosine similarity for text data
    ),
    hdbscan_model=HDBSCAN(
        min_cluster_size=60,             # Increase to reduce topic count
        min_samples=15,                   # Fewer samples to prevent fragmentation
        metric='euclidean',              # Works well with UMAP output
        cluster_selection_method='eom'   # Keeps distinct clusters
    ),                       # Target topic count, reduces smaller ones
    top_n_words=10                       # Top words per topic
)
topics, probs = model.fit_transform(docs)

In [ ]:
print(topics)
print(model.get_topic_info())
uniq_topics = set(topics)
for topic in uniq_topics:
  print(f"topic:{topic}, name:{model.get_topic(topic)}")

In [ ]:
import pandas as pd

topic_info_df = model.get_topic_info()
topic_info_df.to_csv("bertopic_topic_info.csv", index=False)

In [ ]:
df = pd.DataFrame({
    "document": docs,
    "topic": topics,
    "probability": probs
})
df.to_csv("document_topic_mapping.csv", index=False)

In [ ]:
print(max(topics))